In [ ]:
%pip install split-folders 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [ ]:
#check for duplicate pictures
import os
import hashlib

hashes = {}

for type_folder in os.listdir('dataset-classification'):
    for tumor_folder in os.listdir(f'classification/{type_folder}'):
        for file in os.listdir(f'classification/{type_folder}/{tumor_folder}'):
            filepath = f'classification/{type_folder}/{tumor_folder}/{file}'
            with open(filepath, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
                if file_hash not in hashes:
                    hashes[file_hash] = []
                
                hashes[file_hash].append(filepath)
#delete duplicates
count = 0
for file_hash, path_list in hashes.items():
    while len(path_list) > 1: 
        to_delete = path_list.pop()
        os.remove(to_delete)
print(count)


0


In [ ]:
#count dataset images
def count_images(path: str)-> str:
    result = {}
    for type_folder in os.listdir(f'{path}'):
        temp = []
        for tumor_folder in os.listdir(f'{path}/{type_folder}'):
            count = 0
            for file in os.listdir(f'{path}/{type_folder}/{tumor_folder}'):
                count+=1
            temp.append(list([tumor_folder,count]))
        result[type_folder] = temp

    for i in result.keys():
        print(f'{i}: {result[i]}')
count_images('dataset-classification')
#with duplicates:
# Testing: [['glioma_tumor', 400], ['meningioma_tumor', 421], ['no_tumor', 510], ['pituitary_tumor', 374]]
# Training: [['glioma_tumor', 2147], ['meningioma_tumor', 2161], ['no_tumor', 1990], ['pituitary_tumor', 2284]]
# Testing count: 1705
# Training count: 8582

Training: [['glioma_tumor', 2328], ['meningioma_tumor', 1635], ['no_tumor', 1708], ['pituitary_tumor', 1776]]


In [ ]:
#merge testing and training folders (unmerged from original dataset)
import shutil
if os.path.exists(f'dataset-classification/Testing'):
    for tumor_folder in os.listdir(f'dataset-classification/Testing'):
        for file in os.listdir(f'dataset-classification/Testing/{tumor_folder}'):
            shutil.move(f'dataset-classification/Testing/{tumor_folder}/{file}', f'dataset-classification/Training/{tumor_folder}/{file}')
    shutil.rmtree('dataset-classification/Testing')

#split testing folder into testing,validation folders
import splitfolders

input_folder = os.path.join('dataset-classification/Training')
output_folder = os.path.join('dataset_split_classification')
split_ratio = (0.7,0.15,0.15)

splitfolders.ratio(
    input_folder,
    output = output_folder,
    seed = 12,
    ratio = split_ratio,
    group_prefix= None
)

In [ ]:
count_images('dataset_split_classification')

test: [['glioma_tumor', 350], ['meningioma_tumor', 246], ['no_tumor', 257], ['pituitary_tumor', 267]]
train: [['glioma_tumor', 1629], ['meningioma_tumor', 1144], ['no_tumor', 1195], ['pituitary_tumor', 1243]]
val: [['glioma_tumor', 349], ['meningioma_tumor', 245], ['no_tumor', 256], ['pituitary_tumor', 266]]


In [ ]:
#balance dataset
import random
balanced_values = {
    'train' : 1140,
    'val' : 245,
    'test' : 256
}
for type_folder in os.listdir('dataset_split_classification'):
    for tumor_folder in os.listdir(f'dataset_split_classification/{type_folder}'):
        while len(os.listdir(f'dataset_split_classification/{type_folder}/{tumor_folder}')) > balanced_values[type_folder]:
                files = os.listdir(f'dataset_split_classification/{type_folder}/{tumor_folder}')
                os.remove(f'dataset_split_classification/{type_folder}/{tumor_folder}/{random.choice(files)}')

#todo pridat ze cisla balanced_values neni hardcoded ale vytahne to nejnizsi cislo souboru ve slozce + celkove vsechny path nebudou tak hardcoded napr dataset_split/

In [ ]:
count_images('dataset_split_classification')

test: [['glioma_tumor', 256], ['meningioma_tumor', 246], ['no_tumor', 256], ['pituitary_tumor', 256]]
train: [['glioma_tumor', 1140], ['meningioma_tumor', 1140], ['no_tumor', 1140], ['pituitary_tumor', 1140]]
val: [['glioma_tumor', 245], ['meningioma_tumor', 245], ['no_tumor', 245], ['pituitary_tumor', 245]]
